## Using TensorFlow like NumPy
### Tensors and Operations
#### Tensors

In [3]:
import tensorflow as tf

t = tf.constant([[1, 2, 3], [4, 5, 6]])  # datatype int. For float use . after digits
t

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [6]:
display(t.shape)
display(t.dtype)

TensorShape([2, 3])

tf.int32

#### Indexing

In [8]:
t[:,1:]

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2, 3],
       [5, 6]])>

In [15]:
# 3 dots are necessary. Returns coulumn matrix at index 2. Without tf,newaxis returns a row matrix
t[..., 2, tf.newaxis]       

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[3],
       [6]])>

#### Ops

In [17]:
display(t + 10)
display(tf.square(t))  # equivalent to tf.math.square
display(t @ tf.transpose(t))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[11, 12, 13],
       [14, 15, 16]])>

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 1,  4,  9],
       [16, 25, 36]])>

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[14, 32],
       [32, 77]])>

### Tensors and NumPy

In [23]:
import numpy as np

a = np.array([2., 4., 5.])
tf.constant(a)

display(t.numpy())
display(np.array(t))

display(tf.square(a))
display(np.square(t))

array([2., 4., 6.])

array([2., 4., 6.])

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

array([ 4., 16., 36.])

### Type Conversions
***TensorFlow
does not perform any type conversions automatically:***

In [24]:
try:
    tf.constant(2.0) + tf.constant(40)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]


In [25]:
try:
    tf.constant(2.0) + tf.constant(40., dtype=tf.float64)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]


In [26]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### Variables

In [30]:
v = tf.Variable([[1.,2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [31]:
display(v.assign(2 * v))
display(v[0, 1].assign(42))
display(v[:, 2].assign([1., 2.]))
display(v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.]))
sparse_data = tf.IndexedSlices(values=[[1., 2., 3.], [4., 5., 6.]], indices=[1, 0])
display(v.scatter_update(sparse_data))

try:
    v[1] = [7., 8., 9.]
except TypeError as ex:
    print(ex)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  1.],
       [ 8., 10.,  2.]], dtype=float32)>

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   1.],
       [  8.,  10., 200.]], dtype=float32)>

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[4., 5., 6.],
       [1., 2., 3.]], dtype=float32)>

'ResourceVariable' object does not support item assignment


## Custom loss function

In [32]:
import matplotlib.pyplot as plt

class Huber(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs) -> None:
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = tf.abs(y_true - y_pred)
        is_small_error = error < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * error - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold':self.threshold}

In [ ]:
# model = tf.keras.models.load_model("my_model_with_a_custom_loss_class",
                                #    custom_objects={"HuberLoss": HuberLoss})

## Custom Metrics
### Streaming metrics

In [ ]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold ** 2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold, name=None, dtype=None, **kwargs):
        super().__init__(name, dtype, **kwargs)
        self.threshold  = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight('total', initializer='zeros')
        self.count = self.add_weight('count', initializer='zeros')

    def update_state(self, y_true, y_valid, *args, **kwargs):
        sample_metrics = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(sample_metrics))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count

    def get_config(self):
        return {**super().get_config(), 'threshold':self.threshold}

This class handles shapes better, and it also supports sample weights.

In [ ]:
class HuberMetric(tf.keras.metrics.Mean):
    def __init__(self, threshold=1.0, name='HuberMetric', dtype=None):
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        super().__init__(name=name, dtype=dtype)

    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        super(HuberMetric, self).update_state(metric, sample_weight)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}       

## Custom Layers
If you
want to create a custom layer without any weights, the simplest option is to write a function and wrap it in a `tf.keras.layers.Lambda`.

In [33]:
exponential_layer = tf.keras.layers.Lambda(lambda x: tf.exp(x))

# extra code – like all layers, it can be used as a function:
exponential_layer([-1., 0., 1.])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.36787945, 1.        , 2.7182817 ], dtype=float32)>

In [34]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation, trainable=True, name=None, dtype=None, dynamic=False, **kwargs):
        super().__init__(trainable, name, dtype, dynamic, **kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        self.weights = self.add_weight(name='kernel', shape=[input_shape[-1], self.units], initializer='he_normal')
        self.bias = self.add_weight(name='bias', shape=[self.units], initializer='zeros')
        return super().build(input_shape)    # must be at the end

    def call(self, inputs, *args, **kwargs):
        return self.activation(inputs @ self.weights + self.bias)

    def compute_output_shape(self, input_shape):
        return tf.TensorShape(input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        return {**super().get_config(), 'units':self.units, 'activation':self.activation}

## Custom Models

In [ ]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, n_layers, n_nuerons, trainable=True, name=None, dtype=None, dynamic=False, **kwargs):
        super().__init__(trainable, name, dtype, dynamic, **kwargs)
        self.hidden = [tf.keras.layers.Dense(n_nuerons, activation='relu', kernel_initializer='he_normal') 
                        for _ in range(n_layers)]

    def call(self, inputs, *args, **kwargs):
        z = inputs
        for layer in self.hidden:
            z = layer(z)
        return inputs + z

In [35]:
class ResidualRegressor(tf.keras.Model):
    def __init__(self, output_dim, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.hidden = tf.keras.layers.Dense(30, activation='relu', kernel_initializer='he_normal')
        self.block1 = ResidualBlock(2, 30)
        self.block1 = ResidualBlock(2, 30)
        self.output = tf.keras.layers.Dense(output_dim)

    def call(self, inputs, training=None, mask=None):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

## Losses and Metrics Based on Model Internals

In [36]:
class ReconstructingRegressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(30, activation="relu",
                                             kernel_initializer="he_normal")
                       for _ in range(5)]
        self.out = tf.keras.layers.Dense(output_dim)
        self.reconstruction_mean = tf.keras.metrics.Mean(
            name="reconstruction_error")

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = tf.keras.layers.Dense(n_inputs)
        self.built = True  # WORKAROUND for super().build(batch_input_shape)

    def call(self, inputs, training=None):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)
        if training:
            result = self.reconstruction_mean(recon_loss)
            self.add_metric(result)
        return self.out(Z)